In [1]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import os
import pytorch_lightning as pl
import torch

from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.loggers import WandbLogger

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import WeightedRandomSampler
import numpy as np

from ResNet_2 import Net,BasicBlock


In [2]:
import wandb

def Start_wandb(num_epochs=2,architecture="Nice_model"):
    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="FireRed_pixels_classification",

        # track hyperparameters and run metadata
        config={
        #"learning_rate": 0.02,
        "architecture": architecture,
        "dataset": "Pokemon_FireRed_Pixels",
        "epochs": num_epochs,
        }
    )

In [3]:
# Define the data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(16),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.02, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

# Assuming the images are in a directory named "data", with two subdirectories "train" and "val"
data_dir = 'Data'
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
#dataloaders = {x: DataLoader(image_datasets[x], batch_size=3225, shuffle=True,num_workers=8) for x in ['train', 'val']}



# Get class distribution
class_counts = np.bincount(image_datasets['train'].targets)

# Calculate weights
weights = 1. / torch.tensor(class_counts, dtype=torch.float)
sample_weights = weights[image_datasets['train'].targets]

# Create sampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Init our model
model = Net(BasicBlock, [2,2,2,2])

# Init DataLoader from training set
batch_size = 1024
num_workers =  0
train_loader = DataLoader(image_datasets["train"], batch_size=batch_size,sampler=sampler)#,num_workers=num_workers)
#We can actually fit the whole dataset in memory for validation, we don´t do it for training as augmentation takes a toll on the cpu
val_loader = DataLoader(image_datasets["val"], batch_size=batch_size, shuffle=True)#,num_workers=num_workers)

version=len(os.listdir(f'logs/{model.name}'))+10

In [4]:
wandb.finish()

In [5]:
checkpoint = torch.load('checkpoints/ResNet_1_17/version_68/best-checkpoint.ckpt')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [6]:
torch.set_float32_matmul_precision('medium')
wandb.init(
        # set the wandb project where this run will be logged
        project="FireRed_pixels_classification",

        # track hyperparameters and run metadata
        config={
        #"learning_rate": 0.02,
        "architecture": "ResNet_1_17",
        "dataset": "Pokemon_FireRed_Pixels",
        "epochs": 100000,
        }
    )

early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=500,
    mode="min",
)
#We wanna optimize precision (we wanna miss as little as possible)
checkpoint_callback = ModelCheckpoint(
    monitor="val_precision",
    dirpath=f"checkpoints/{model.name}/version_{version}",
    filename="best-checkpoint",
    save_top_k=1,
    mode="max",
)

checkpoint_callback2 = ModelCheckpoint(
    monitor="val_acc",
    dirpath=f"checkpoints/{model.name}/version_{version}",
    filename="best-checkpoint",
    save_top_k=1,
    mode="max",
)
checkpoint_callback3 = ModelCheckpoint(
    monitor="val_auroc",
    dirpath=f"checkpoints/{model.name}/version_{version}",
    filename="best-checkpoint",
    save_top_k=1,
    mode="max",
)

Start_wandb(num_epochs=10000,architecture=model.name)


wandb_logger = WandbLogger(savedir='logs',name=model.name,version=version)
logger_csv = CSVLogger("logs", name=model.name,version=version)
#log_every_n_steps=10 es para quitar el warning porque realmente no llegamos a 50, sino que cada epoch~13 steps
trainer = pl.Trainer(logger=[wandb_logger,logger_csv],max_epochs=10000, devices=1, accelerator="gpu",callbacks=[early_stopping_callback,checkpoint_callback,checkpoint_callback2,checkpoint_callback3])#,log_every_n_steps=10)
trainer.fit(model,train_loader,val_loader)
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jaimerbaron. Use `wandb login --relogin` to force relogin


c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | conv1  | Conv2d           | 1.7 K 
1 | bn1    | BatchNorm2d      | 128   
2 | layer1 | Sequential       | 147 K 
3 | layer2 | Sequential       | 525 K 
4 | layer3 | Sequential       | 2.1 M 
5 | layer4 | Sequential       | 8.4 M 
6 | linear | Linear           | 8.7 K 
7 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.

Sanity Checking: 0it [00:00, ?it/s]

c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\Rafa\miniconda3\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
c:\Users\Rafa\miniconda3\lib\site-packages\torchmetrics\utilities\prints.py:36: UserW

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▄▅▆▅█▇▆▆▇▅▆▅▅▄▂▄▃▇▅▃▄▃▃▆▃▄▃▂▅▅▆▅▃▄▆▅▅▄▃▁
train_acc_class0,█▆▆▆▇█▆██▄▄▆▆▆█▆▇█▆▆▅▂▅▁█▇█▅▄▇▆▅▅▆███▃█▆
train_acc_class1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc_class10,▆▄██▇▆█▄██▄█▆████▆█▆█▄▆▆▅█▆▄▁▄▆▆█▄██▅█▆▄
train_acc_class11,▄▇▇▇▇▆▆▆█▆█▇▆▅▁▅▂▅▅▁▅▅▆▇▂▂▄▅▅▆▅▄▆▃▅▅▅▂▃▂
train_acc_class12,▁█████████████▂████▅█▅███▄██████▄█████▅█
train_acc_class13,████████████████▁██▃████▁██▁███████▃▃███
train_acc_class14,██████████████▄███████▁█████████▄██████▂
train_acc_class15,██████████▁███▄██████▂███████████▄██▃█▃█
train_acc_class16,███████████████████▁████████████████████


In [7]:
#early_stopping_callback = EarlyStopping(
#    monitor="val_loss",
#    patience=50,
#    mode="min",
#)
##We wanna optimize precision (we wanna miss as little as possible)
#checkpoint_callback = ModelCheckpoint(
#    monitor="val_precision",
#    dirpath=f"checkpoints/{model.name}/version_{version}",
#    filename="best-checkpoint",
#    save_top_k=1,
#    mode="max",
#)
#Start_wandb(num_epochs=10000,architecture=model.name)


In [8]:
wandb.finish()

In [9]:
from torch import nn
